In [2]:
using JuMP
using Gurobi
using CSV 
using DataFrames

In [3]:
df = CSV.read("data.csv", DataFrame)
D_t = df[!, "FR_load_forecast_entsoe_transparency"]
# println(size(D_t))


file_path = "capacity_pv.csv"
df = CSV.File(file_path; header=true) |> DataFrame
capacity_pv = Array(df)
println(first(capacity_pv, 10))
    
file_path = "capacity_wind.csv"
df = CSV.File(file_path; header=true) |> DataFrame
capacity_wind = Array(df)
println(first(capacity_wind, 10))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.067, 0.218]
[0.12594, 0.123693, 0.124232, 0.13045, 0.140045, 0.136766, 0.129185, 0.135791, 0.152821, 0.14136]


In [4]:
# Constants and Data
T = 8760
lambda = 5000  
cost_of_debt = 0.04
cost_of_equity = 0.07
corporate_tax = 0.30
economic_life = 20

capacity = Dict(
    "Coal" => 1,
    "CCGT" => 1,
    "OCGT" => 1,
    "Onshore Wind" => capacity_wind,
    "Offshore Wind" => capacity_wind,
    "PV" => capacity_pv
)

# Technologies data
technologies = Dict(
    "Coal" => (capex=2000, om=0.03, debt_ratio=0.62, heat_rate=2.4, EA=122.657651, price =5, C=12),
    "CCGT" => (capex=950, om=0.03, debt_ratio=0.56, heat_rate=1.62, EA=59.92311601, price =30, C=48.6),
    "OCGT" => (capex=700, om=0.03, debt_ratio=0.6, heat_rate=2.5, EA=43.33807688, price =30, C=75),
    "Onshore Wind" => (capex=700, om=0.03, debt_ratio=0.7, heat_rate=0, EA=41.29858169, price =0, C=0),
    "Offshore Wind" => (capex=1300, om=0.03, debt_ratio=0.7, heat_rate=0, EA=76.69736599, price =0, C=0),
    "PV" => (capex=400, om=0.03, debt_ratio=0.8, heat_rate=0, EA=22.43376371, price =0, C=0)
)

#Battery data
F_b=6
A_b=1.620627574
eta=0.9

model_2= Model(Gurobi.Optimizer)


@variable(model_2, k[g in keys(technologies)] >= 0)  # Installed capacity
@variable(model_2, p[g in keys(technologies), t=1:T] >= 0)  # Hourly production
@variable(model_2, d[t=1:T] >=0)
#For Battery
@variable(model_2, k_b>=0 ) 
@variable(model_2, e[g in keys(technologies),1:T] >= 0)
@variable(model_2, p_in[g in keys(technologies), 1:T] >=0)
@variable(model_2, p_out[g in keys(technologies), 1:T] >=0)


# @objective(model_2, Min, sum(k[g] for g in keys(technologies)))
# @objective(model_2, Min, sum( (technologies[g].om * technologies[g].capex + technologies[g].EA )*k[g] + (A_b + F_b)* k_b + sum(technologies[g].price*p[g,t]  for t in 1:T) for g in keys(technologies))- sum( lambda * d[t] for t in 1:T))
@objective(model_2, Min, sum((technologies[g].om * technologies[g].capex + technologies[g].EA )*k[g] + sum(technologies[g].C*p[g,t]  for t in 1:T) for g in keys(technologies))+ (A_b + F_b)* k_b - sum( lambda * d[t] for t in 1:T))

@constraint(model_2, [ g in keys(technologies), t in 1:T], p[g,t] <= k[g])
@constraint(model_2, [t in 1:T], d[t]<=D_t[t])
@constraint(model_2, [t in 1:T], d[t]- sum( p[g,t] + p_out[g,t] - p_in[g,t] for g in keys(technologies))==0)
@constraint(model_2, [g in keys(technologies), t in 1:T], e[g,t]<= 4*k_b)
@constraint(model_2, [g in keys(technologies), t in 1:T], p_in[g,t]<=k_b)
@constraint(model_2, [g in keys(technologies), t in 1:T], p_out[g,t]<=k_b)
@constraint(model_2, [g in keys(technologies)], e[g,1]==0)
@constraint(model_2, [g in keys(technologies),t in 2:T], e[g,t]== e[g,t-1] + t*(sqrt(eta)* p_in[g,t-1] - 1/sqrt(eta) * p_out[g,t-1]))
for g in ["Coal", "CCGT", "OCGT"]
    @constraint(model_2, [t in 1:T], p[g, t] <= k[g])
end
for t in 1:T
    @constraint(model_2, p["Onshore Wind", t] <= capacity_wind[t]*k["Onshore Wind"])
    @constraint(model_2, p["Offshore Wind", t] <= capacity_wind[t]*k["Offshore Wind"])
    @constraint(model_2, p["PV", t] <= capacity_pv[t]*k["PV"])
end

optimize!(model_2)
# print("Variables value, p:",JuMP.value.(p))
print("Variables value, k:",JuMP.value.(k))
print("Variables value, k_b:",JuMP.value.(k_b))
# print("Variables value, d:",JuMP.value.(d))
print("Objective value: ", objective_value(model_2))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-07
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 332880 rows, 219007 columns and 906916 nonzeros
Model fingerprint: 0xbe635583
Coefficient statistics:
  Matrix range     [1e-03, 9e+03]
  Objective range  [8e+00, 5e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+04, 1e+05]
Presolve removed 43273 rows and 12881 columns
Presolve time: 1.27s
Presolved: 289607 rows, 206126 columns, 820367 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.15s

Barrier statistics:
 Dense cols : 7
 AA' NZ     : 2.598e+05
 Factor NZ  : 1.798e+06 (roughly 90 MB of memory)
 Factor Ops : 2.996e+07 (less than 1 second per iteration)
 Threads 